# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests



# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
0,0,vrangel',42.7297,133.0839,37.85,83,100,10.42,RU,1711576405
1,1,port-aux-francais,-49.3500,70.2167,42.66,85,100,29.19,TF,1711576405
2,2,west island,-12.1568,96.8225,80.58,78,100,8.05,CC,1711576406
3,3,adamstown,-25.0660,-130.1015,76.57,65,35,12.80,PN,1711576406
4,4,ushuaia,-54.8000,-68.3000,47.86,57,0,9.22,AR,1711576382


 Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
#%%capture --no-display
plot_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True, 
    tiles = 'OSM', 
    frame_width = 700, 
    frame_height = 500, 
    color='City',
    alpha= 0.5,
    s='Humidity',
    scale= 0.8)

# Display the map
plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [16]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cond_df = city_data_df.loc[((city_data_df["Max Temp"] <= 30.0) & (city_data_df["Max Temp"] >= 20.0)) & (city_data_df["Cloudiness"] == 0),:]

# Drop any rows with null values
ideal_cond_df = ideal_cond_df.dropna(how="any")


# Display sample data
ideal_cond_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
78,78,selfoss,63.9331,-20.9971,27.28,57,0,13.60,IS,1711576421
155,155,vadso,70.0744,29.7487,23.16,68,0,10.36,NO,1711576435
293,293,brookings,44.3114,-96.7984,27.18,66,0,18.41,US,1711576444
297,297,flin flon,54.7682,-101.8650,24.64,33,0,4.61,CA,1711576189
303,303,bismarck,46.8083,-100.7837,26.44,51,0,9.22,US,1711576465


### Step 3: Create a new DataFrame called `hotel_df`.

In [17]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cond_df[["City","Country","Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()


C:\Users\em\AppData\Local\Temp\ipykernel_38132\2949095225.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
78,selfoss,IS,63.9331,-20.9971,57,
155,vadso,NO,70.0744,29.7487,68,
293,brookings,US,44.3114,-96.7984,66,
297,flin flon,CA,54.7682,-101.8650,33,
303,bismarck,US,46.8083,-100.7837,51,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [19]:
# Set parameters to search for a hotel
radius = 1000
params = {
    "apiKey":geoapify_key,
    "limit":10,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
selfoss - nearest hotel: Miðgarður
vadso - nearest hotel: Scandic Vadsø
brookings - nearest hotel: No hotel found
flin flon - nearest hotel: No hotel found
bismarck - nearest hotel: Radisson Hotel Bismarck


,City,Country,Lat,Lng,Humidity,Hotel Name
78,selfoss,IS,63.9331,-20.9971,57,Miðgarður
155,vadso,NO,70.0744,29.7487,68,Scandic Vadsø
293,brookings,US,44.3114,-96.7984,66,No hotel found
297,flin flon,CA,54.7682,-101.8650,33,No hotel found
303,bismarck,US,46.8083,-100.7837,51,Radisson Hotel Bismarck


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
%%capture --no-display
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=650,
    frame_height=500,
    size="Humidity",
    color="City",
    hover_cols=["Hotel Name","Country"]
    )
# Configure the map plot
# YOUR CODE HERE

# Display the map
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)